In [1]:
def getInput():
    return open("../input/day_5.txt").read()

def getInputLines():
    return getInput().split("\n")

# Day 5 Part 1

parsing: /`(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)`/gms

In [3]:
import re

text = getInput()

# text = """
# seeds: 79 14 55 13

# seed-to-soil map:
# 50 98 2
# 52 50 48

# soil-to-fertilizer map:
# 0 15 37
# 37 52 2
# 39 0 15

# fertilizer-to-water map:
# 49 53 8
# 0 11 42
# 42 0 7
# 57 7 4

# water-to-light map:
# 88 18 7
# 18 25 70

# light-to-temperature map:
# 45 77 23
# 81 45 19
# 68 64 13

# temperature-to-humidity map:
# 0 69 1
# 1 0 69

# humidity-to-location map:
# 60 56 37
# 56 93 4
# """.strip()

# match all numbers, first match is seed line, subsequent matches are the maps
seeds, *maps = [
    [int(v) for v in re.findall(r"\d+", match.groups()[1], flags=re.MULTILINE)]
    for match in re.finditer(r"(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)", text, flags=re.MULTILINE)
    ]
# sort ranges in maps based on their start number
maps = [sorted(r, key=lambda x: x[1]) for r in
    [[m[i:i+3] for i in range(0, len(m), 3)] for m in maps]]

# main
locs = []
for seed in seeds:
    temp = seed
    for map in maps:
        for (dst, src, span) in map:
            if src <= temp < src + span:
                temp = dst + (temp - src)
                break
    locs.append(temp)
min(locs)
# 331445006


331445006

# Day 5 Part 2

- 24092691 too high
- 20522066 too high
- 19624706 too high

In [4]:
import re

text = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()

text = getInput()

seeds, *maps = [[int(v) for v in re.findall(r"\d+", match.groups()[1], flags=re.MULTILINE)] for match in re.finditer(r"(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)", text, flags=re.MULTILINE)]
seeds = [seeds[i:i+2] for i in range(0, len(seeds), 2)]
maps = [[m[i:i+3] for i in range(0, len(m), 3)] for m in maps]

for_next_map = [seed for seed in seeds]
for i, rules in enumerate(maps):
    for_next_rule = [seed for seed in for_next_map]
    for_next_map = []
    for (r_dst, r_start, r_len) in rules:
        for_this_rule = [seed for seed in for_next_rule]
        for_next_rule = []
        for (s_start, s_len) in for_this_rule:
            r_shift = r_dst - r_start
            r_end = r_start + r_len - 1
            s_end = s_start + s_len - 1
            if s_end < r_start or s_start > r_end: # no overlap
                for_next_rule.append((s_start, s_len))
            elif s_start >= r_start and s_end <= r_end: # seed range is fully included in rule
                for_next_map.append((s_start + r_shift, s_len))
            elif s_start >= r_start: # seed range ends after rule
                for_next_map.append((s_start + r_shift, r_len))
                for_next_rule.append((r_end + 1, s_end - r_end + 1))
            elif s_end <= r_end: # seed range starts before rule
                for_next_map.append((r_start + r_shift, r_len))
                for_next_rule.append((s_start, r_start - s_start + 1 ))
            else: # seed range fully contains rule
                for_next_map.append((r_start + r_shift, r_len))
                for_next_rule.append((s_start, r_start - s_start))
                for_next_rule.append((r_end + 1, s_end-r_end+1))        
            # print(i, for_next_rule)
    for_next_map += for_next_rule
    print(i, for_next_map)
print("result is:", min(for_next_map, key=lambda x: x[0])[0])


0 [(81, 14), (57, 13)]
1 [(81, 14), (57, 13)]
2 [(53, 8), (81, 14), (61, 10)]
3 [(46, 8), (74, 14), (54, 10)]
4 [(45, 23), (82, 8), (90, 10), (78, 13), (77, 2)]
5 [(46, 23), (82, 8), (90, 10), (78, 13), (77, 2)]
6 [(60, 37), (86, 8), (94, 37), (82, 13), (81, 2), (56, 4), (46, 11), (97, 5)]
result is: 46
